In [ ]:
import ROOT

in_file = ROOT.TFile("~/work/sti-profiling/inputs_joinTwo.root")

t = in_file.Get("t")

ROOT.enableJSVis()
ROOT.gStyle.SetPadRightMargin(0.3)

In [ ]:
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

In [ ]:
# Define some functions

g_h_counter = 1

# Fill 1D histogram from the tree
def project1D(expression, n_bins, x_first, x_last, h_title = "", condition = ""):
    
    global g_h_counter
    
    h_name = 'h_{:03d}'.format(g_h_counter)
    g_h_counter += 1
    
    if h_title == "":
        h_title = expression
    
    h1D = ROOT.TH1D(h_name, h_title, n_bins, x_first, x_last)
    
    t.Project(h_name, expression, condition)
    
    return h1D



def drawOnCanvases(hists):
    
    canvases = []
    
    for hist in hists:
        canvas = ROOT.TCanvas("canvas_" + hist.GetName(), "ROOT Canvas", 800, 600)
        hist.Draw()
        canvas.Draw()
        canvases.append(canvas)
        
    return canvases



def plotly1D(h1D):

    y_values_ptr = h1D.GetArray()
    y_values = []
    x_values = []
    
    for i_bin in range( 1, h1D.GetNbinsX() + 1 ):
        y_values.append( y_values_ptr[i_bin] )
        x_values.append( h1D.GetBinCenter(i_bin) )

    #return [ go.Bar(y = y_values) ]
    plotly.offline.iplot( [ go.Bar(y = y_values) ] )

In [ ]:
hists = []

# Create and plot input distributions
hists.append( project1D("P1[0]", 100, -20, 200, "; x; events", "nP1 == 6") )
hists.append( project1D("P1[1]", 100, -60, 60, "; y; events", "nP1 == 6") )
hists.append( project1D("P1[2]", 100, -250, 250, "; z; events", "nP1 == 6") )
hists.append( project1D("P1[3]", 100, -1.5, 1.5, "; #phi; events", "nP1 == 6") )
hists.append( project1D("P1[4]", 100, -100, 100, "; q/Pt; events", "nP1 == 6") )
hists.append( project1D("P1[5]", 100, -5, 5, "; tan(#Lambda); events", "nP1 == 6") )

canvases = drawOnCanvases(hists)

In [ ]:
hists = []

hists.append( project1D("fabs(P2[0] - PJ[0]) / sqrt(E2[1])", 50, 0, 10) )
hists.append( project1D("fabs(P2[1] - PJ[1]) / sqrt(E2[2])", 50, 0, 10) )
hists.append( project1D("fabs(P2[2] - PJ[2]) / sqrt(E2[5])", 50, 0, 10) )
hists.append( project1D("fabs(P2[3] - PJ[3]) / sqrt(E2[9])", 50, 0, 10) )
hists.append( project1D("fabs(P2[4] - PJ[4]) / sqrt(E2[14])", 50, 0, 10) )
hists.append( project1D("fabs(P2[5] - PJ[5]) / sqrt(E2[20])", 50, 0, 10) )

canvases = drawOnCanvases(hists)

In [ ]:
diff_P2_PJ_chi2 = ( 
  " (P2[1] - PJ[1]) * (P2[1] - PJ[1]) / E2[2]  + "
  " (P2[2] - PJ[2]) * (P2[2] - PJ[2]) / E2[5]  + "
  " (P2[3] - PJ[3]) * (P2[3] - PJ[3]) / E2[9]  + "
  " (P2[4] - PJ[4]) * (P2[4] - PJ[4]) / E2[14] + "
  " (P2[5] - PJ[5]) * (P2[5] - PJ[5]) / E2[20]   "
)

hists = []

hist = project1D(diff_P2_PJ_chi2, 50, 0, 50, "; #chi^2; ", "nP1 == 6")
hist.Scale( 1./hist.Integral() )
hist = hist.GetCumulative()
hists.append( hist )

canvases = drawOnCanvases(hists)


In [ ]:
#trace0 = [ go.Bar(y = values_list) ]

#plotly.offline.iplot(trace0, filename = 'basic-line')